In [1]:
import pandas as pd
import numpy as np
import os
#os.chdir("../")

In [2]:
no_precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="reszta")
precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="Prekambr")

### NIE-PREKAMBR - SYSTEM ERY - single - easy

In [54]:
temp=no_precambr_data[["ERA", "SYSTEM"]].drop_duplicates()
n = len(temp)

In [60]:
target_list = temp["ERA"].unique()
target_list = [ele for ele in target_list for i in range(n*2)]
scope_list = temp["SYSTEM"].unique()

In [77]:
question_text = []
possible_answers = []
correct_answers = []

for target in target_list:
    question_text.append(f"Wypisz system ery {target}")
    correct_array = temp[temp["ERA"]==target]["SYSTEM"].unique()
    wrong_array = [element for element in scope_list if element not in correct_array]

    # print(f"CORRECT {correct_array}")
    # print(f"WRONG {wrong_array}")
    # if len(wrong_array)<4:
    #     print('WARNING!!!!!-------------------')
    #     possible_answers.append(np.repeat(np.nan, 5))
    #     correct_answers.append(np.nan)
    #     continue

    temp_possible_answers = np.concatenate(
        (np.random.choice(wrong_array, 3, replace=False), np.random.choice(correct_array, 1))
    )
    np.random.shuffle(temp_possible_answers)
    temp_possible_answers = np.append(temp_possible_answers, np.nan)
    for correct in temp_possible_answers:
        if correct in correct_array:
            correct_answers.append(correct)
            continue

    possible_answers.append(temp_possible_answers)

df1=pd.DataFrame({
    'question':question_text,
    'correct_answer':correct_answers
})

df2=pd.DataFrame(
    possible_answers,
    columns=["A", "B", "C", "D", "E"]
)

df_final = pd.concat([df1, df2], axis=1).drop_duplicates()

In [78]:
df_final

,question,correct_answer,A,B,C,D,E
0,Wypisz system ery kenozoik,czwartorzęd,czwartorzęd,kambr,karbon,trias,NaN
1,Wypisz system ery kenozoik,paleogen,ordowik,perm,paleogen,kambr,NaN
2,Wypisz system ery kenozoik,paleogen,perm,kreda,kambr,paleogen,NaN
3,Wypisz system ery kenozoik,paleogen,trias,jura,karbon,paleogen,NaN
4,Wypisz system ery kenozoik,paleogen,sylur,kreda,paleogen,jura,NaN
...,...,...,...,...,...,...,...
67,Wypisz system ery paleozoik,dewon,dewon,kreda,trias,czwartorzęd,NaN
68,Wypisz system ery paleozoik,dewon,kreda,neogen,paleogen,dewon,NaN
69,Wypisz system ery paleozoik,karbon,karbon,jura,paleogen,neogen,NaN
70,Wypisz system ery paleozoik,kambr,paleogen,trias,kambr,kreda,NaN


### MULTI

In [21]:
temp=no_precambr_data[["ERA", "SYSTEM"]].drop_duplicates()
n = len(temp)
target_list = temp["ERA"].unique()
target_list = [ele for ele in target_list for i in range(n*2)]
scope_list = temp["SYSTEM"].unique()

In [53]:
question_text = []
wrong_answers = []
correct_answers = []
possible_answers = []

for target in target_list:
    question_text.append(f"Wypisz system ery {target}")
    correct_array = temp[temp["ERA"]==target]["SYSTEM"].unique()
    wrong_array = [element for element in scope_list if element not in correct_array]

    # print(f"CORRECT {correct_array}")
    # print(f"WRONG {wrong_array}")
    
    # if len(wrong_array)<4:
    #     print('WARNING!!!!!-------------------')
    #     possible_answers.append(np.repeat(np.nan, 5))
    #     correct_answers.append(np.nan)
    #     continue

    if len(correct_array)>5:
        randomSampleCorrectSize = np.random.randint(1, 6)
    else:
        randomSampleCorrectSize = np.random.randint(1,len(correct_array))

    wrongSample = 5-randomSampleCorrectSize

    temp_wrong_answers = np.random.choice(wrong_array, wrongSample, replace=False)
    temp_correct_answers = np.random.choice(correct_array, randomSampleCorrectSize, replace=False)
    temp_possible_answers = np.concatenate((temp_correct_answers, temp_wrong_answers))
    np.random.shuffle(temp_possible_answers)
    possible_answers.append(temp_possible_answers)

    correct_answers.append(temp_correct_answers)

df1=pd.DataFrame({
    'question':question_text,
    'correct_answer':correct_answers
})

df2=pd.DataFrame(
    possible_answers,
    columns=["A", "B", "C", "D", "E"]
)

df_final = pd.concat([df1, df2], axis=1)

In [54]:
df_final

,question,correct_answer,A,B,C,D,E
0,Wypisz system ery kenozoik,"[neogen, czwartorzęd]",czwartorzęd,kambr,neogen,sylur,trias
1,Wypisz system ery kenozoik,[neogen],sylur,perm,kambr,karbon,neogen
2,Wypisz system ery kenozoik,"[paleogen, neogen]",paleogen,sylur,kreda,neogen,dewon
3,Wypisz system ery kenozoik,"[paleogen, neogen]",ordowik,trias,dewon,neogen,paleogen
4,Wypisz system ery kenozoik,"[neogen, czwartorzęd]",neogen,czwartorzęd,sylur,kreda,perm
...,...,...,...,...,...,...,...
67,Wypisz system ery paleozoik,"[ordowik, perm]",trias,jura,kreda,ordowik,perm
68,Wypisz system ery paleozoik,"[karbon, sylur, dewon, kambr]",dewon,karbon,sylur,czwartorzęd,kambr
69,Wypisz system ery paleozoik,[karbon],kreda,karbon,jura,neogen,czwartorzęd
70,Wypisz system ery paleozoik,"[kambr, ordowik]",jura,kambr,neogen,ordowik,paleogen


In [2]:
from question_builder import QuestionBuilder

no_precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="reszta")
precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="Prekambr")

test = QuestionBuilder(no_precambr_data, precambr_data)

In [24]:
dfs = []
criteria = [
    (False, True, True, 1, 9, "ERA", "SYSTEM", "Wybierz system ery", []),
    (False, True, False, 1, 9, "ERA", "PIETRO", "Wybierz piętro ery", []),
    (False, True, False, 1, 9, "SYSTEM", "ODDZIAL", "Wybierz system oddziału", ['kreda','jura','trias','dewon','ordowik']),
    (False, True, False, 1, 9, "SYSTEM", "PIETRO", "Wybierz piętro systemu", []),
    (False, True, False, 1, 9, "ODDZIAL", "PIETRO", "Wybierz piętro oddziału", []),
    # prek
    (True, True, True, 1, 9, "ERA", "SYSTEM", "Wybierz system ery", ['neoarchaik','mezoarchaik','paleoarchaik','eoarchaik','brak']),
]

n = 0
for is_prek, if_multi, both_scopes, multiplyIdx, possAns, targetName, scopeName, questionTxt, explusionList in criteria:
    temp = test.prepare_closed_ended(is_prek, if_multi, both_scopes, multiplyIdx, possAns, targetName, scopeName, questionTxt, explusionList)
    dfs.append(temp)
    n+=1
    print(n)

1
2
3
4
5
6


In [25]:
dfs[5]

,question,correct_answer,A,B,C,D,E,F,G,H,I
0,Wybierz system ery neoproterozoik,[ton],sten,dewon,ordowik,czwartorzęd,orosir,brak,ton,karbon,paleogen
1,Wybierz system ery neoproterozoik,"[ton, kriogen]",kalim,dewon,riak,stater,kreda,trias,perm,ton,kriogen
2,Wybierz system ery neoproterozoik,[ediakar],ediakar,kambr,kreda,trias,kalim,jura,paleogen,perm,sylur
3,Wybierz system ery neoproterozoik,[kriogen],trias,sider,ordowik,ektas,kambr,riak,kreda,kriogen,neogen
4,Wybierz system ery neoproterozoik,"[ediakar, kriogen]",sten,trias,kreda,ediakar,sylur,kriogen,sider,dewon,karbon
5,Wybierz system ery neoproterozoik,"[kriogen, ton]",kambr,kriogen,sten,jura,perm,ektas,kreda,ton,orosir
6,Wybierz system ery neoproterozoik,[ton],czwartorzęd,dewon,stater,sider,ton,kalim,ordowik,orosir,kreda
7,Wybierz system ery neoproterozoik,[ediakar],czwartorzęd,perm,sider,kalim,kreda,kambr,stater,ediakar,jura
8,Wybierz system ery neoproterozoik,[ton],perm,riak,stater,kambr,ton,karbon,brak,kreda,czwartorzęd
9,Wybierz system ery neoproterozoik,"[kriogen, ton]",brak,sider,karbon,trias,ton,kriogen,neogen,kreda,sylur


In [6]:
df = test.prepare_closed_ended(True, True, True, 10, 6 ,"ERA", "SYSTEM", "Wybierz system ery", ["brak", "eoarchaik", "paleoarchaik", "mezoarchaik", "neoarchaik"])

,question,correct_answer,A,B,C,D,E,F
0,Wybierz system ery neoproterozoik,"[ton, ediakar]",sylur,perm,ediakar,sider,ton,jura
1,Wybierz system ery neoproterozoik,[ediakar],kambr,kalim,stater,kreda,orosir,ediakar
2,Wybierz system ery neoproterozoik,"[ton, kriogen]",kambr,stater,jura,kriogen,ton,paleogen
3,Wybierz system ery neoproterozoik,"[ton, ediakar]",karbon,jura,riak,ediakar,ton,brak
4,Wybierz system ery neoproterozoik,[ton],ordowik,ton,sider,stater,jura,dewon
...,...,...,...,...,...,...,...,...
445,Wybierz system ery paleoproterozoik,[sider],kambr,kreda,ordowik,trias,paleogen,sider
446,Wybierz system ery paleoproterozoik,"[orosir, sider, stater]",orosir,kalim,ektas,jura,stater,sider
447,Wybierz system ery paleoproterozoik,"[riak, stater, orosir]",stater,riak,neogen,kambr,brak,orosir
448,Wybierz system ery paleoproterozoik,"[orosir, riak, stater]",orosir,kreda,riak,dewon,stater,ediakar


In [4]:
dfs = []
criteria = [
    # NON PRECAMBRIAN
    # era
    (False, False, 3, "ERA", "SYSTEM", "Wybierz system ery"),
    (False, True, 15, "ERA", "SYSTEM", "Wybierz system ery"),
    (False, False, 3, "ERA", "ODDZIAL", "Wybierz oddział ery"),
    (False, True, 15, "ERA", "ODDZIAL", "Wybierz oddział ery"),
    (False, False, 3, "ERA", "PIETRO", "Wybierz piętro ery"),
    (False, True, 15, "ERA", "PIETRO", "Wybierz piętro ery"),
    # system
    (False, False, 3, "SYSTEM", "ODDZIAL", "Wybierz oddział systemu"),
    (False, True, 15, "SYSTEM", "ODDZIAL", "Wybierz oddział systemu"),
    (False, False, 3, "SYSTEM", "PIETRO", "Wybierz piętro systemu"),
    (False, True, 15, "SYSTEM", "PIETRO", "Wybierz piętro systemu"),
    # oddział
    (False, False, 3, "ODDZIAL", "PIETRO", "Wybierz piętro oddziału"),
    (False, True, 15, "ODDZIAL", "PIETRO", "Wybierz piętro oddziału"),

    ## PRECAMBRIAN
    # prekambr
    #(True, False, 2, "Jednostka nieformalna", "Eon", "Prekambr - Wybierz eon prekambru"),
    (True, True, 10, "Jednostka nieformalna", "Eon", "Prekambr - Wybierz eon prekambru"),
    (True, False, 2, "Jednostka nieformalna", "Era", "Prekambr - Wybierz erę prekambru"),
    (True, True, 10, "Jednostka nieformalna", "Era", "Prekambr - Wybierz erę prekambru"),
    (True, False, 2, "Jednostka nieformalna", "System", "Prekambr - Wybierz system prekambru"),
    (True, True, 10, "Jednostka nieformalna", "System", "Prekambr - Wybierz system prekambru"),
    # eon
    (True, False, 2, "Eon", "Era", "Prekambr - Wybierz erę enou"),
    (True, True, 10, "Eon", "Era", "Prekambr - Wybierz erę enou"),
    (True, False, 2, "Eon", "System", "Prekambr - Wybierz system enou"),
    (True, True, 10, "Eon", "System", "Prekambr - Wybierz system enou"),
    # system ery
    (True, False, 2, "Era", "System", "Prekambr - Wybierz system ery"),
    (True, True, 10, "Era", "System", "Prekambr - Wybierz system ery"),

]
n = 0
for is_prek, is_hard, coeff, key1, key2, quest_txt in criteria:
    temp = test.prepare_single(is_prek, is_hard, coeff, key1, key2, quest_txt)
    dfs.append(temp)
    n+=1
    print(n)
    print(f"{is_prek} {is_hard} {coeff} {key1}, {key2}, {quest_txt}")

12
1
False False 3 ERA, SYSTEM, Wybierz system ery
172
2
False True 15 ERA, SYSTEM, Wybierz system ery
34
3
False False 3 ERA, ODDZIAL, Wybierz oddział ery
172
4
False True 15 ERA, ODDZIAL, Wybierz oddział ery
102
5
False False 3 ERA, PIETRO, Wybierz piętro ery
172
6
False True 15 ERA, PIETRO, Wybierz piętro ery
34
7
False False 3 SYSTEM, ODDZIAL, Wybierz oddział systemu
172
8
False True 15 SYSTEM, ODDZIAL, Wybierz oddział systemu
102
9
False False 3 SYSTEM, PIETRO, Wybierz piętro systemu
172
10
False True 15 SYSTEM, PIETRO, Wybierz piętro systemu
102
11
False False 3 ODDZIAL, PIETRO, Wybierz piętro oddziału
172
12
False True 15 ODDZIAL, PIETRO, Wybierz piętro oddziału
172
13
True True 10 Jednostka nieformalna, Eon, Prekambr - Wybierz eon prekambru
8
WARNING!!!!!-------------------
[]
3


ValueError: 

In [4]:
c = (True, True, 5, "Jednostka nieformalna", "Eon", "Prekambr - Wybierz eon prekambru")
df1 = test.prepare_single(c[0], c[1], c[2], c[3], c[4], c[5])

['aalen' 'aeron' 'akwitan' 'alb' 'anizyk' 'apt' 'archaik' 'artyńsk'
 'assel' 'bajos' 'barrem' 'barton' 'baszkir' 'baton' 'berrias' 'brak'
 'burdygał' 'cenoman' 'chiban' 'cisural' 'czangsing' 'czwartorzęd' 'dan'
 'daping' 'darriwil' 'dewon' 'dewon dolny' 'dewon górny' 'dewon środkowy'
 'drum' 'dżangszan' 'ediakar' 'eifel' 'ektas' 'ems' 'eoarchaik' 'eocen'
 'epoka 2' 'famen' 'floj' 'fortun' 'fran' 'furong' 'gelas' 'gorst'
 'grenland' 'gużang' 'gwadelup' 'gżel' 'hadeik' 'hettang' 'hirnant'
 'holocen' 'homer' 'hoteryw' 'ind' 'iprez' 'jura' 'jura dolna'
 'jura górna' 'jura środkowa' 'kalabr' 'kalim' 'kambr' 'kampan' 'kapitan'
 'karbon' 'karnik' 'kasim' 'kat' 'kelowej' 'kenozoik' 'kimeryd' 'koniak'
 'kreda' 'kreda dolna' 'kreda górna' 'kriogen' 'kungur' 'ladyn' 'landower'
 'lang' 'lochkow' 'loping' 'ludford' 'ludlow' 'lutet' 'mastrycht'
 'megalaj' 'messyn' 'mezoarchaik' 'mezoproterozoik' 'mezozoik' 'miaoling'
 'miocen' 'mississip' 'moskow' 'neoarchaik' 'neogen' 'neoproterozoik'
 'northgrip' 

In [5]:
df1

,question,correct_answer,A,B,C,D,E
0,Prekambr - Wybierz eon prekambru prekambr,archaik,hirnant,archaik,kreda górna,dan,tremadok
1,Prekambr - Wybierz eon prekambru prekambr,hadeik,hadeik,paleoproterozoik,gużang,ems,chiban
2,Prekambr - Wybierz eon prekambru prekambr,archaik,artyńsk,northgrip,road,sakmar,archaik
3,Prekambr - Wybierz eon prekambru prekambr,archaik,trias środkowy,pensylwan,zankl,archaik,mississip
4,Prekambr - Wybierz eon prekambru prekambr,proterozoik,cisural,piętro 2,proterozoik,kreda górna,gużang
5,Prekambr - Wybierz eon prekambru prekambr,archaik,sten,orosir,plejstocen,gorst,archaik
6,Prekambr - Wybierz eon prekambru prekambr,archaik,piętro 4,synemur,loping,lutet,archaik
7,Prekambr - Wybierz eon prekambru prekambr,archaik,priabon,kreda,stater,alb,archaik
8,Prekambr - Wybierz eon prekambru prekambr,proterozoik,piacent,mezoarchaik,fortun,chiban,proterozoik
9,Prekambr - Wybierz eon prekambru prekambr,hadeik,santon,hadeik,aalen,baszkir,paleocen


In [5]:
# df_easy = test.prepare_single(False, False, 3, "ERA", "SYSTEM", "Wybierz system ery")
# df_hard = test.prepare_single(False, True, 5, "ERA", "SYSTEM", "Wybierz system ery")

,question,correct_answer,A,B,C,D,E
0,Wybierz system ery kenozoik,neogen,trias,neogen,kreda,perm,NaN
1,Wybierz system ery kenozoik,czwartorzęd,dewon,czwartorzęd,kambr,jura,NaN
2,Wybierz system ery kenozoik,czwartorzęd,dewon,karbon,czwartorzęd,kambr,NaN
3,Wybierz system ery kenozoik,neogen,jura,neogen,sylur,kambr,NaN
4,Wybierz system ery kenozoik,czwartorzęd,dewon,perm,czwartorzęd,kreda,NaN
...,...,...,...,...,...,...,...
103,Wybierz system ery paleozoik,ordowik,jura,czwartorzęd,ordowik,neogen,NaN
104,Wybierz system ery paleozoik,sylur,czwartorzęd,paleogen,jura,sylur,NaN
105,Wybierz system ery paleozoik,kambr,kreda,trias,paleogen,kambr,NaN
106,Wybierz system ery paleozoik,ordowik,neogen,ordowik,jura,czwartorzęd,NaN


In [3]:
import pandas as pd
import numpy as np
import os

In [20]:
non_prec_single = pd.read_csv(fr"{os.getcwd()}\outputs\non_precambrian_questions_single.csv")
non_prec_multi = pd.read_csv(fr"{os.getcwd()}\outputs\non_precambrian_questions_multi.csv")
prec_single = pd.read_csv(fr"{os.getcwd()}\outputs\precambrian_questions_single.csv")
prec_multi = pd.read_csv(fr"{os.getcwd()}\outputs\precambrian_questions_multi.csv")

In [26]:
dfs = [
    non_prec_single,
    non_prec_multi,
    prec_single,
    prec_multi
]

In [31]:
n=0
new_dfs = []
for df in dfs:
    temp = df.copy()
    temp = temp.drop_duplicates()
    temp['isHard'] = np.where(pd.isnull(temp['E']), 0, 1)

    if n%2==0:
        temp['isMulti'] = 0
    else:
        temp['isMulti'] = 1

    temp = temp.reset_index()
    new_dfs.append(temp)
    n+=1

In [35]:
df_final = pd.concat(new_dfs)

In [38]:
df_final.to_csv(fr"{os.getcwd()}\outputs\all_questions_main_df.csv")